In [14]:
from selenium import webdriver
import time, os
import io
import requests
from bs4 import BeautifulSoup
from PIL import Image
import hashlib

In [2]:
# When migrating away from Jupyter notebook, this will proabbly be moved to a __main__
DRIVER_PATH = "./chromedriver"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [3]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)
        
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    
    wd.get(search_url.format(q=query))
    
    image_urls = set()
    image_count = 0
    results_start = 0
    
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)
        
        thumbnail_results = wd.find_elements_by_css_selector("img.rg_ic")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue
                
            actual_images = wd.find_elements_by_css_selector('img.irc_mi')
            for actual_image in actual_images:
                if actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
                    
            image_count = len(image_urls)
            
            if image_count >= max_links_to_fetch:
                print(f"Found: {image_count} image links, done!")
                break
            else:
                print("Found:", image_count, "image links, looking for more...")
                time.sleep(1)
                load_more_button = wd.find_element_by_css_selector(".ksb")
                if load_more_button:
                    wd.execute_script("document.querySelector('.ksb').click();")
                    
            results_start = len(thumbnail_results)
    return image_urls

In [4]:
fetch_image_urls("pelican", 5, wd)

Found: 200 search results. Extracting links from 0:200
Found: 2 image links, looking for more...
Found: 3 image links, looking for more...
Found: 4 image links, looking for more...
Found: 5 image links, done!


{'https://download.ams.birds.cornell.edu/api/v1/asset/70589261/1800',
 'https://nhpbs.org/wild/images/whitepelicanjohnfosterusfw.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/1/1f/Pelecanus_conspicillatus_-_Doughboy_Head.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/d/d4/Pelikan_Walvis_Bay.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/75/Australian_Pelican_showing_large_pouch.jpg/220px-Australian_Pelican_showing_large_pouch.jpg'}

In [4]:
# wd.quit()

In [10]:
def persist_image(folder_path:str, url:str):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - could not download {url} - {e}")
        
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path, hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [6]:
def search_and_download(search_term:str, driver_path:str, target_path='./images', number_images=5):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))
    
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder, elem)

In [16]:
search_term = "capybara"

search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

Found: 200 search results. Extracting links from 0:200
Found: 1 image links, looking for more...
Found: 2 image links, looking for more...
Found: 3 image links, looking for more...
Found: 4 image links, looking for more...
Found: 5 image links, done!
SUCCESS - saved https://cdn.theatlantic.com/assets/media/img/photo/2018/11/photos-companionable-capybaras/c01_890874902-1/original.jpg - as ./images/capybara/ac2fe0197b.jpg
SUCCESS - saved https://www.nationalgeographic.com/content/dam/animals/thumbs/rights-exempt/mammals/c/capybara.jpg - as ./images/capybara/cb8ab7371e.jpg
SUCCESS - saved https://animals.sandiegozoo.org/sites/default/files/2016-10/animals_hero_capybara.jpg - as ./images/capybara/4763ed2363.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/e/ec/Capybara_%28Hydrochoerus_hydrochaeris%29.JPG - as ./images/capybara/89341033c0.jpg
SUCCESS - saved https://www.rainforest-alliance.org/sites/default/files/styles/750w_585h/public/2016-09/capybara.jpg?itok=_4AI2DG6 -